In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.format('xml').options(rowTag='page').load('hdfs:/enwiki_small.xml')

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-02c5d77a-8205-445b-8fa6-f1ea9139a07b;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 294ms :: artifacts dl 8ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [3]:
df.show()

+---+--------+------------+--------------------+--------------------+
| id|redirect|restrictions|            revision|               title|
+---+--------+------------+--------------------+--------------------+
| 10|        |        null|{{[[Help:Revertin...| AccessibleComputing|
| 12|    null|        null|{{Undid revision ...|           Anarchism|
| 13|        |        null|{{cat rd, null}, ...|  AfghanistanHistory|
| 14|        |        null|{{1 revision from...|AfghanistanGeography|
| 15|        |        null|{{Robot: Fixing [...|   AfghanistanPeople|
| 18|        |        null|{{cat rd, null}, ...|AfghanistanCommun...|
| 19|        |        null|{{cat rd, null}, ...|AfghanistanTransp...|
| 20|        |        null|{{cat rd, null}, ...| AfghanistanMilitary|
| 21|        |        null|{{{{R from CamelC...|AfghanistanTransn...|
| 23|        |        null|{{cat rd, null}, ...| AssistiveTechnology|
| 24|        |        null|{{cat rd, null}, ...|        AmoeboidTaxa|
| 25|    null|      

In [4]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- redirect: string (nullable = true)
 |-- restrictions: string (nullable = true)
 |-- revision: struct (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |-- contributor: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- username: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- minor: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _xml:space: string (nullable = true)
 |    |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)



In [5]:
import regex
from pyspark.sql.functions import lower, udf, col, explode

def extractLink(text):
    try:
        results = regex.findall(r'\[\[((?:[^[\]]+|(?R))*+)\]\]', text)
    except:
        results = []
    output = []
    for res in results:
        for link in res.split('|'):
            if ':' in link and 'Category:' not in link:
                continue
            elif '#' in link:
                continue
            else:
                output.append(link.lower())
                break
    return output

In [10]:
from pyspark.sql.types import StringType, ArrayType

link_udf = udf(lambda text: extractLink(text), ArrayType(StringType()))
newdf = df.withColumn("article", explode(link_udf(col("revision.text._VALUE"))))

In [7]:
newdf.printSchema()

root
 |-- id: long (nullable = true)
 |-- redirect: string (nullable = true)
 |-- restrictions: string (nullable = true)
 |-- revision: struct (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |-- contributor: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- username: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- minor: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _xml:space: string (nullable = true)
 |    |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- article: string (nullable = true)



In [12]:
newdf = newdf.select(lower(col('title')).alias('title'), 'article').orderBy('title', 'article')

In [ ]:
newdf.repartition(10).write.option("delimiter", "\t").csv('p2t2_small')